Option 2 - Write a computer program in your favorite programming language to perform the angular spectrum method for calculating the diffraction pattern fro an arbitrary nearfield pattern. 

In [37]:
#Required Libraries
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog

In [38]:
#initialize Variables
lamb = 390*10**(-9)
t = 1/lamb
omega = 2*np.pi/t
k = 2*np.pi/lamb
c = omega/k
#Distance of measurement away from mask (in m)
z = 0.05
gain = 0.01


In [39]:
#Import mask image file
file = filedialog.askopenfilename(initialdir = '/Users/junju/OneDrive/School/Homework/Year 2/Semester 2/ESE 438/HW6/Input Images', title = 'Select Mask Image')
mask = cv2.imread(file,0)
print(mask.shape)
print("Image file: " + file + " has been loaded")


(512, 512)
Image file: C:/Users/junju/OneDrive/School/Homework/Year 2/Semester 2/ESE 438/HW6/Input Images/Untitled.png has been loaded


In [40]:
pixWidthX = len(mask[0])
pixWidthY = len(mask)

In [41]:
#Function to convert wavelength to 
def wavelength_to_rgb(wavelength, gamma=0.8):

    wavelength = float(wavelength)
    if wavelength >= 380 and wavelength <= 440:
        attenuation = 0.3 + 0.7 * (wavelength - 380) / (440 - 380)
        R = ((-(wavelength - 440) / (440 - 380)) * attenuation) ** gamma
        G = 0.0
        B = (1.0 * attenuation) ** gamma
    elif wavelength >= 440 and wavelength <= 490:
        R = 0.0
        G = ((wavelength - 440) / (490 - 440)) ** gamma
        B = 1.0
    elif wavelength >= 490 and wavelength <= 510:
        R = 0.0
        G = 1.0
        B = (-(wavelength - 510) / (510 - 490)) ** gamma
    elif wavelength >= 510 and wavelength <= 580:
        R = ((wavelength - 510) / (580 - 510)) ** gamma
        G = 1.0
        B = 0.0
    elif wavelength >= 580 and wavelength <= 645:
        R = 1.0
        G = (-(wavelength - 645) / (645 - 580)) ** gamma
        B = 0.0
    elif wavelength >= 645 and wavelength <= 750:
        attenuation = 0.3 + 0.7 * (750 - wavelength) / (750 - 645)
        R = (1.0 * attenuation) ** gamma
        G = 0.0
        B = 0.0
    else:
        R = 0.0
        G = 0.0
        B = 0.0
    R *= 255
    G *= 255
    B *= 255
    #B and R inverted for OpenCV compatibility
    return (int(B), int(G), int(R))

In [42]:
#Angular Spectrum Method Function
fft_m = np.fft.fft2(mask)
m = np.fft.fftshift(fft_m)

#Spatial Frequency * 2 * Pi for kx and ky
kx = 2*np.pi*np.fft.fftshift(np.fft.fftfreq(pixWidthX, d = (5.6*10**(-3))/pixWidthX))
ky = 2*np.pi*np.fft.fftshift(np.fft.fftfreq(pixWidthY, d = (5.6*10**(-3))/pixWidthY))
kx, ky = np.meshgrid(kx, ky)
kz = np.sqrt(k**2 - kx**2 - ky**2)



In [43]:
#Inverse FFT to return to original form
newM = np.fft.ifft2(np.fft.ifftshift(m * np.exp(1j * kz * z)))

#Calculate intensity of resulting plane
intensity = np.real(newM * np.conjugate(newM)) * gain

#Convert Monochrome Image into RGB (Still retaining original color)
intensity = np.array(intensity, dtype = np.float32)
colorFormatIntensity = cv2.cvtColor(intensity ,cv2.COLOR_GRAY2RGB)

#Find color of monochromatic plane wave
colorRGB = wavelength_to_rgb(lamb*10**9)

#Apply previously calculated color value while retaining previous intensity value
for i in range(pixWidthX):
    for j in range(pixWidthY):
        for k in range(0, 3):
            colorFormatIntensity[i][j][k] = colorRGB[k] *colorFormatIntensity[i][j][k]/255
cv2.imwrite('/Users/junju/OneDrive/School/Homework/Year 2/Semester 2/ESE 438/AngularSpectrumMethodGUI/Output Images/new.png', colorFormatIntensity )

True

In [44]:
noEvanescent = np.where(kz**2 > 0, m * np.exp(1j * kz * z), 0)
noEvM = np.fft.ifft2(np.fft.ifftshift(noEvanescent))
noEvIntensity = np.real(noEvM * np.conjugate(noEvM))* gain
noEvIntensity = np.array(noEvIntensity, dtype = np.float32)
noEvColorFormatIntensity = cv2.cvtColor(noEvIntensity ,cv2.COLOR_GRAY2RGB)
for i in range(pixWidthX):
    for j in range(pixWidthY):
        for k in range(0, 3):
            noEvColorFormatIntensity[i][j][k] = colorRGB[k] *noEvColorFormatIntensity[i][j][k]/255
cv2.imwrite('/Users/junju/OneDrive/School/Homework/Year 2/Semester 2/ESE 438/AngularSpectrumMethodGUI/Output Images/newnoEV.png', noEvIntensity)

True